In [ ]:
import sys,os
repo_path = '/home/debian/'
sys.path.append(repo_path)
sys.path.append(os.path.join(repo_path, 'simodd-pop-scabies'))
sys.path.append(os.path.join(repo_path, 'simodd-dis-scabies'))
sys.path.append(os.path.join(repo_path, 'simodd-dis-scabies/scabies-transmission-Monrovia'))

In [ ]:
#https://stackoverflow.com/questions/6974695/python-process-pool-non-daemonic
import multiprocessing
# We must import this explicitly, it is not imported by the top-level
# multiprocessing module.
import multiprocessing.pool

class NoDaemonProcess(multiprocessing.Process):
    # make 'daemon' attribute always return False
    def _get_daemon(self):
        return False
    def _set_daemon(self, value):
        pass
    daemon = property(_get_daemon, _set_daemon)

# We sub-class multiprocessing.pool.Pool instead of multiprocessing.Pool
# because the latter is only a wrapper function, not a proper class.
class MyPool(multiprocessing.pool.Pool):
    Process = NoDaemonProcess

In [ ]:
from population.utils import create_path

from disease.general.contact_matrix import KnownContactMatrix
from disease.general.contact_matrix import ContactMatrix
from disease.general.contact_matrix import ContactMatrix2

from disease.general.run import go_single
from disease.SEIR.disease_SEIR import DiseaseSIS, DiseaseSIS_NoRecCases
from disease.observers.obs_disease import DiseaseObserver

from disease.experiments.param_combo import ParamComboIt
from disease.observers.obs_summary_stats_sample import SummaryStatsObserver3

from params import p

import pandas as pd
import numpy as np
np.set_printoptions(threshold=sys.maxsize)
from disease.general.mda import VaccineSingle,VaccineHH

from itertools import product
from itertools import repeat

import h5py    
import numpy as np  
import matplotlib.pyplot as plt
import re

#halve dur Infection by the intervention
from disease.general.duration import DurationGenerator
from disease.SEIR.states_SEIR import *


In [ ]:
def try_go_single(p, disease_type, cmatrix, cur_seed):
    try:
        go_single(p, disease_type, cmatrix, cur_seed)
    except:
        global problematic_params_sets
        problematic_params_sets.append(p)
        pass

In [ ]:
#select the parameter set from 'output/generated_q_parameter_sets/'
infestation_duration = 90 #in days
samples = np.genfromtxt('output/generated_q_parameter_sets/qh_q_%s.csv'%infestation_duration,delimiter=',')
samples = np.array([list(x) for x in set(tuple(x) for x in samples)])
display(len(samples))

In [ ]:
#intervention_multipliers_dur_I : multiplier of duration of infestation after last MDA round
intervention_multipliers_dur_I = [1]#[0.8,0.9,1] 

total_vaccine_rounds_list = [3]
vaccine_coverage_list= [0.8]#[0.6,0.7,0.8,0.9]
treatment_fail_probability_list = [0.9]
spill_over_list = [0] #0 spillover: no scabies importation 
mda_selection_list = [VaccineSingle]
vaccine_time_intervals_list = [1]
noncompliance_percentage_list = [0] 
combo_scenarios= list(product(total_vaccine_rounds_list,vaccine_coverage_list,
                       treatment_fail_probability_list,spill_over_list,
                               vaccine_time_intervals_list,mda_selection_list,intervention_multipliers_dur_I,
                             noncompliance_percentage_list))

#age limits for intervention
min_age = 0
max_age = 150
no_vaccines = 1
immediately_effective = False


if __name__ == '__main__':

    for cur_scenario in combo_scenarios:
        total_vaccine_rounds = cur_scenario[0]
        vaccine_coverage = cur_scenario[1]
        fail_prob = cur_scenario[2]
        spill_over = cur_scenario[3]
        vaccine_time_interval = cur_scenario[4]
        mda_type = cur_scenario[5]
        intervention_multiplier_dur_I = cur_scenario[6]
        noncompliance_percentage = cur_scenario[7]
        intervention_points = np.arange(1, 1 + vaccine_time_interval*total_vaccine_rounds, vaccine_time_interval).tolist()
        p['vaccination_t'] = [(int(p['burn_in']+p['epi_burn_in'])  + i)* p['t_per_year'] for i in intervention_points]
        individual_vaccination_per_year = 1
        vaccination_intervals_in_weeks = (np.diff(intervention_points)*p['t_per_year']).tolist()
        individual_offset_year = intervention_points
        if total_vaccine_rounds:
            offset_year = individual_offset_year[0]
            p['offset_year'] = offset_year
            p['individual_offset_year'] = [offset_year]
        
            vaccination_saving_address = "%s_vaccines_%s_coverage_%s_rounds_%s_intervals_%s_fail_%s_%s_age_%s_offset_delay_%s_%s_spill_%s_selection_Int_%s_noncompliance_%s"%(
                                                                                            no_vaccines,
                                                                                         vaccine_coverage,total_vaccine_rounds,
                                                                                         vaccination_intervals_in_weeks,
                                                                                         fail_prob,min_age,max_age,
                                                                                         offset_year,1-int(immediately_effective),spill_over,
                                                                                    re.split("Vaccine|'",str(mda_type))[2], intervention_multiplier_dur_I,
                                                                                        noncompliance_percentage)
        else:
            vaccination_saving_address = "No_intervention_Int_%s"%intervention_multiplier_dur_I
        display(vaccination_saving_address)
        p['total_vaccine_rounds'] = total_vaccine_rounds
        p['vaccine_coverage'] = vaccine_coverage
        p['fail_prob'] = fail_prob
        p['spill_over'] = spill_over
        p['vaccine_time_interval'] = vaccine_time_interval
        p['mda_type'] = mda_type
        p['intervention_points'] = intervention_points
        p['individual_vaccination_per_year'] = individual_vaccination_per_year
        p['individual_offset_year'] = individual_offset_year
        p['noncompliance_percentage'] = noncompliance_percentage
        
        display(index)
        

        class DiseaseModel(DiseaseSIS_NoRecCases):
            """
            Local version of SIR disease, adding observers and vaccines specific 
            to this set of experiments.
            """

            def __init__(self, p, cmatrix, rng, fname, mode='w'):
                super(DiseaseModel, self).__init__(p, cmatrix, rng, fname, mode)

                self.add_observers(
                    SummaryStatsObserver3(h5file=self.h5file,fname=fname, #file name for csv documents
                                          rng=rng,
                             observefrom=(int(0 + p['burn_in']) * p['t_per_year']),#observe results from that time point
                             et=int((p['burn_in'] +p['epi_burn_in']+ p['years'][1]) * p['t_per_year']), #end time of simulation to save csv files
                             t_per_year=p['t_per_year']),
                    DiseaseObserver(self.h5file, self.state_labels(),
                                    self.disease_states),
                    )    

                if no_vaccines >= 1:
                    for j in range(p['total_vaccine_rounds']):
                        self.add_vaccines(mda_type(label="Vaccine%s"%j, min_age = 0, max_age = 150, age_days=364, 
                                                coverage=[p['vaccine_coverage']], 
                                                t_per_year=p['t_per_year'], 
                                                offset=int(p['burn_in']+p['epi_burn_in']) + p['individual_offset_year'][j], #rng=rng, #rng=Random(p['seed']),
                                                 noncompliance_percentage = p['noncompliance_percentage'],
                                                   precond=None, fail_prob = p['fail_prob'], 
                                                v_state='S' ,vaccination_per_year=p['individual_vaccination_per_year'],
                                                immediately_effective = False))
                else:
                    vaccination_t =  [(1500)* p['t_per_year']]
                    vaccination_saving_address = "No_intervention_Int_%s"%intervention_multiplier_dur_I
                    
            def update(self, t, P, rng):
                """
                Update the disease state of the population.
                """
                     
                self.check_vaccines(t, P, rng)
                # print "(update)", introduction

                cases = self.check_exposure(t, P, rng)

                introductions = self.external_exposure(t, P, rng)

                # print "(update)", introduction
                new_I = self.update_ind_states(t, P)
                #display("%s: %s, I %s"%(t, self.states['I'].duration.theta,len(new_I)))
                
                self.update_observers(t, disease=self, pop=P,
                                      cases=cases['infection'],
                                      boosting=cases['boosting'],
                                      introductions=introductions,
                                      new_I=new_I, rng=rng)


                if t == (int(p['burn_in']+p['epi_burn_in']) + p['individual_offset_year'][-1])* p['t_per_year']:
                    display("t:%s, %s"%(t,self.states['I'].duration.theta))
                    self.states['I'].duration.theta = self.states['I'].duration.theta * intervention_multiplier_dur_I
                    display("t:%s, %s"%(t,self.states['I'].duration.theta))
                    
                # if halting, halt if no exposed or infected individuals in population (eg for SIR)
                return self.halt and not sum([self.states[x].count for x in self.disease_states])


    # - # - # - # - # - # - # - # - # - # - # - # - # - # - # - # - # - # - # - #
    # - # - MAIN  - # - #
    # - # - # - # - # - # - # - # - # - # - # - # - # - # - # - # - # - # - # - #


        # setup base parameters
        p['pop_prefix'] = 'output/burn_in_population/'
        p['epi_prefix'] = 'output/burn_in_epidemics/spillover_%s'%(spill_over)
        
        epi_directory = os.path.join(repo_path, 'simodd-dis-scabies/example',p['epi_prefix'])
        if not os.path.exists(epi_directory):
            os.makedirs(epi_directory)
        p['prefix'] = 'output/mda_scenarios/' + vaccination_saving_address + '/'
        p['epi_burn_in'] = 10
        p['halt'] = False
        p['overwrite'] = True
        p['overwrite_cp'] = True
        p['save_cp'] = True
        p['years']= [0, 21] #simulation time
        p['external_exposure_rate'] = spill_over
        # number of different seeds to use for each parameter combination
        p['num_runs'] = 1
        
        
        sweep_params_list=[]
        all_param_combos=[]
        for s in range(len(samples[:,0])):
            # sweep parameters
            sweep_params = [
                {'name': 'q', 'values': [(samples[:,1][s])]},
                {'name': 'q_h', 'values': [(samples[:,0][s])]},
                {'name': 'infective_mean', 'values': [(infestation_duration)]},
            ]
            
            sweep_params_list.append(sweep_params)
            # generate parameter combinations (converting iterator to list)
            param_combos = list(ParamComboIt(p, sweep_params))
            param_combos[0]['external_exposure_rate'] = param_combos[0]['external_exposure_rate'] * 13635 *\
                                                        param_combos[0]['q']
            all_param_combos.append(param_combos[0])
            
        display(param_combos[0]['external_exposure_rate'])
        liberia_matrix = pd.read_csv(os.path.join(repo_path, 'simodd-dis-scabies/example/data/Liberia_contact_matrix.csv'),header=None).to_numpy()
        cmatrix = ContactMatrix2(given_matrix=liberia_matrix,smooth=False)
        dmodel = DiseaseModel

        seeds=[item['seed'] for item in all_param_combos]    #[param_combos[i::noprocess]['seed'] for i in range(noprocess)]
        #run the batch simulations
        mypool = MyPool()#Pool(processes=min(noprocess,batch_size))
        results = mypool.starmap(go_single, zip(all_param_combos,repeat(dmodel),repeat(cmatrix),seeds))
        mypool.close()
        mypool.join()
        
        #OR: run single simulation 
        #go_single(all_param_combos[0],dmodel,cmatrix,seeds[0])
        index +=1